# Résolution d'équation

Pour le réaliser sans installation, depuis un navigateur :
<a href="https://colab.research.google.com/github/eddes/INSA/blob/main/python/tuto_solve_equation.ipynb"> ça se passe ici<a>

On ne distingue pas le cas des équations linéaires/non-linéaires, car la méthode utilisée est construite pour résoudre les deux types (il faut creuser la doc de scipy pour découvrir qu'il s'agit d'un algorithme hybride de Powell). Ceci est discutable, cependant le propos ici est de réussir à se débrouiller pour obtenir le résultat souhaité, sans forcément chercher l'optimum de vitesse : en effet pour les systèmes linéaires, la convergence est sans doute plus rapide avec l'algorithme TDMA.

Ceci étant dit : en avant Guingamp !
    
## Une équation, une inconnue
    
### Principe

Prenons l'exemple de la fonction $x^3 +2 x = 2$. La méthode consiste à toujours mettre l'équation que vous cherchez sous la forme de $f(x)=0$, car les algorithmes de résolution numérique sont conçus pour trouver le "zéro" d'une fonction.
    
Ainsi on va résoudre $x^3 +2 x - 2 = 0$, en créant une fonction dédiée :

In [2]:
# l'import de la librairie qui fait ce genre de choses
from scipy.optimize import  fsolve

# on définit la fonction pour laquelle on "cherche le zero"
def fc_a_resoudre(x):
    return x**3 + 2*x - 2

# on doit donner un point de depart pour la recherche de solution
x_initialisation = 1.5
# et on resout, sans arguments pour la fonction (voir la suite)
x_solution = fsolve(fc_a_resoudre, x_initialisation, args=())
print("x_solution vaut ...", x_solution)

x_solution vaut ... [0.770917]


In [3]:
fc_a_resoudre(0.77)


-0.003466999999999887

Faites la vérification en utilisant `fc_a_resoudre(0.77)` : on devrait trouver une valeur proche de zéro.

Il est également possible (et parfois pratique) d'ajouter des paramètres ou "arguments" pour la fonction à résoudre, par exemple si $x^3 + kx = 2$ avec $k$ connu, défini par l'utilisateur :

In [4]:
# l'import de la librairie qui fait ce genre de choses
from scipy.optimize import  fsolve

# on définit la fonction à résoudre, meme principe
def fc_a_resoudre(x, param):
    return x**3 + param*x - 2

# on doit donner un point de depart pour la recherche de solution
x_initialisation = 1.5

#supposons que le parametre prenne la valeur suivante
param=30

# et on resout, avec arguments cette fois
x_solution = fsolve(fc_a_resoudre, x_initialisation, args=(param))
print("x_solution vaut ...", x_solution)

x_solution vaut ... [0.06665679]


Faites la vérif' comme dans l'exemple précédent.

*Remarque* : Les paramètres à résoudre peuvent être nombreux, il suffit de les ajouter dans  `args=(param1, param2, ...)`.


### Application - Pression de vapeur saturante

<span style="color:green"> *À vous de jouer : la pression de vapeur saturante en fonction de la température est donnée dans la fonction ci-dessous. Trouvez la température pour laquelle $p_{vs} = 2400$ [Pa].* </span>

In [17]:
import numpy as np
# la fonction de pression de vapeur saturante
def pvs(T):
    a,b=0.07252,0.0002881
    c,d=0.00000079,611
    return d * np.exp(a*T -b*np.power(T,2) + c*np.power(T,3))

# il faut definir une fonction telle que "2400- pvs(T) renvoie  0"...

# Réalisé par Val

# l'import de la librairie qui fait ce genre de choses
from scipy.optimize import  fsolve

# on définit la fonction pour laquelle on "cherche le zero"

def fc_a_resoudre(T):
    return -pvs(T)+2400

# on doit donner un point de depart pour la recherche de solution
T_initialisation = 1.5

# et on resout, sans arguments pour la fonction (voir la suite)
T_solution = fsolve(fc_a_resoudre, T_initialisation, args=())
print("T_solution vaut ...", T_solution)



T_solution vaut ... [20.43090026]


## Systèmes d 'équations

La résolution fonctionne exactement sur le même principe, à la différence qu'on va fournir un vecteur de "zéros" à résoudre.

### Principe

Supposons qu'on souhaite résoudre le système de deux équations à deux inconnues $x_a,x_b$ suivant :
$$
\begin{eqnarray}
    x_a^3 + k x_b &=& 2 \\
    x_b^2 - k x_a &=& -6
\end{eqnarray}
$$

In [18]:
# on définit la fonction à intégrer, meme principe
def fc_a_resoudre(x, param):
    xa,xb= x
    zero1 = xa**3 + param*xb - 2
    zero2 = -xb**2 - param*xa + 6

    return np.array([zero1, zero2])

# on doit donner un point de depart pour la recherche de solution
x_initialisation = [1.5, 0]

#supposons que le parametre prenne la valeur suivante
param=2

# et on resout, sans arguments
x_solution = fsolve(fc_a_resoudre, x_initialisation, args=(param))
print("x_solution vaut ...", x_solution)

x_solution vaut ... [ 1.73095309 -1.59313961]


### Application - Production d'oxyde d'azote lors de la combustion de méthane

On souhaite résoudre l'équation de combustion suivante afin de connaître la quantité de NO produit lors de la combustion du méthane. La quantité d'oxydes d'azote produite dépend de la <a href="https://fr.wikipedia.org/wiki/Loi_d%27action_de_masse"> loi d'action de masse<a>, connue grâce à la constante d'équilibre $K_{NO}$.

$$
\begin{eqnarray}
	CH_4 + 2 ( 1 + e ) ( O_2 + 3.76 N_2 )\rightarrow a CO_2 + b H_2O + c O_2 + d NO + f N_2  \\
	\frac{1}{2} N_2 + \frac{1}{2}  O_2  \rightleftarrows  NO
\end{eqnarray}
$$
    
La constante d'équilibre $K_\text{NO}$ étant connue, i s'agit d'équilibrer la stoechiométrie de l'équation (simplifiée) suivante, çàd de trouver les valeurs des coefficients $a, b, c, d, f$, avec $e=0.1$ l'excès d'air :
$$
\begin{eqnarray}
    \text{Équilibre carbone }	a - 1 = 0  \\
    \text{Équilibre oxygène }	2a + b + 2c + d - 4(1+e) = 0\\
	\text{Équilibre hydrogène } 2b -4 = 0  \\
	\text{Équilibre azote } d  +2f - 4 \times 3.76(1+e) = 0\\
	\text{Loi d'action de masse } K_\text{NO} -\frac{d}{ \sqrt{c \times f}}=0
\end{eqnarray}
$$

À vous de jouer, avec l'ébauche ci-dessous :

In [29]:
import numpy as np

# l'import de la librairie qui fait ce genre de choses
from scipy.optimize import  fsolve

# valeur de K_NO
K_NO = 0.2*1e-2

def fc_systeme(x, K_NO):
    a,b,c,d,f = x
    e=0.1

    # je cree un vecteur vide pour contenir les 5 "equations" = 0
    syst_x = np.empty(5)

    # on ecrit les equations
    syst_x[0] = a-1
    syst_x[1] = 2*a+b+2*c+d-4*(1+e)
    syst_x[2] = 2*b-4
    syst_x[3] = d+2*f-4*3.76*(1+e)
    syst_x[4] = K_NO-((d)/(np.sqrt(c*f)))
    return syst_x

    # Réalisé par Val

    # on définit la fonction à résoudre, meme principe

def fc_a_resoudre(x, K_NO):
    a,b,c,d,f = x
    zero0 = syst_x[0]
    zero1 = syst_x[1]
    zero2 = syst_x[2]
    zero3 = syst_x[3]
    zero4 = syst_x[4]

    return np.array([zero0, zero1, zero2, zero3, zero4])

# on doit donner un point de depart pour la recherche de solution
x_initialisation = [1.5, 1, 1, 1, 1]

#supposons que le parametre prenne la valeur suivante
param=K_NO

# et on resout, sans arguments
x_solution = fsolve(fc_a_resoudre, x_initialisation, args=(K_NO))
print("x_solution vaut ...", x_solution)



x_solution vaut ... [1.5 1.  1.  1.  1. ]


<ipython-input-29-1cee1b70706f>:21: RuntimeWarning: invalid value encountered in sqrt
  syst_x[4] = K_NO-((d)/(np.sqrt(c*f)))
<ipython-input-29-1cee1b70706f>:38: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  x_solution = fsolve(fc_a_resoudre, x_initialisation, args=(K_NO))
